In [12]:
import json
import pandas as pd
import sqlite3
import os
from datetime import datetime
import os.path
import time
import sys
import shutil

In [ ]:
'''

tem que ler a landing, pegar a palavra depois do "underline" exemplo: "resultado_teste_", "resultado_concat_"  -> "teste" ou "concat" 
e verificar se o arquivo já foi ingerido
se foi ingerido ignorar 
se não ingerir os novos arquivos

ler o diretório layout e pegar as informações de cada arquivo se caso for necessario a ingestão

'''

In [17]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")

                            # Extraindo informações do dicionário de forma adequada
                            banco = next(info["banco"] for info in tabela_info if "banco" in info)
                            nomeArquivoChave = next(info["nomeArquivoChave"] for info in tabela_info if "nomeArquivoChave" in info)
                            nome_tabela = next(info["tabela"] for info in tabela_info if "tabela" in info)
                            
                            print(f'Nome banco de dados: {banco}')
                            print(f'Nome chave: {nomeArquivoChave}')
                            print(f'Nome tabela: {nome_tabela}')
                            print(f'==============================')
                            
                            with open(ctrl_ing, 'r+') as controle_ingestao:
                                conteudo_load = controle_ingestao.readlines()
                                print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                            
                                for root, dirs, lista in os.walk(landing):
                                    print(f'Procurando em {root}...')
                                    for nome_arquivo in lista:
                                        print(f'Verificando arquivo: {nome_arquivo}')
                                        if nomeArquivoChave and nomeArquivoChave in nome_arquivo:
                                            caminho_completo = os.path.join(root, nome_arquivo)
                                            print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                            sav = os.path.basename(caminho_completo)
                                            print(f'Conteúdo a gravar: {sav}')
                                            
                                            if sav + '\n' not in conteudo_load:
                                                controle_ingestao.write(f"{sav}\n")
                                                print(f'Dado salvo: {sav}')
                                                controle_ingestao.flush()
                                                print('=================================')
                                            
                                            if caminho_completo.endswith('.txt'):
                                                df = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                            elif caminho_completo.endswith('.csv'):
                                                df = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                            elif caminho_completo.endswith('.bin'):
                                                pass
                                            else:
                                                raise ValueError("Formato de arquivo não suportado")
                                            print('Convertendo tipos de dados')
                                            
                                            for col in config_colunas['colunas']:
                                                if col['tipo'] == 'int':
                                                    df[col['nome']] = df[col['nome']].astype(int)
                                                elif col['tipo'] == 'float':
                                                    df[col['nome']] = df[col['nome']].astype(float)
                                                elif col['tipo'] == 'data':
                                                    df[col['nome']] = pd.to_datetime(df[col['nome']], format=col.get('formato', None))
                                            
                                            print('Conectando no banco de dados')
                                            banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                            conn = sqlite3.connect(banco_caminho)
                                            print(f'Banco de dados: {banco_caminho}')
                                            print(f'Conectado!')
                                            cursor = conn.cursor()
                                            
                                            print('Criando tabela no banco de dados...')
                                            sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                            for col in config_colunas['colunas']:
                                                sql_create_table += f"{col['nome']} {col['tipo']}, "
                                            sql_create_table = sql_create_table[:-2] + ")"
                                            cursor.execute(sql_create_table)
                                            
                                            print('Inserindo dados na tabela...')
                                            for index, row in df.iterrows():
                                                mapped_values = []
                                                for col in config_colunas['colunas']:
                                                    mapped_values.append(row.get(col['nome'], None))
                                                print("Valores mapeados:", mapped_values)
                                                sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                print("SQL:", sql)
                                                cursor.execute(sql, mapped_values)

                                            
                                            
                                            print('Commit e fechamento da conexão...')
                                            conn.commit()
                                            conn.close()
                                            print("Ingestão de dados concluída com sucesso!")
                                            
                                            print("========================================")
                                            print("Movimentando arquivos")
                                            shutil.move(caminho_completo, caminho_backup)
                                            print(f'Arquivo enviado para bckp: {caminho_completo}')
                                            print(f'Destino: {caminho_backup}')
                                        
                                        else:
                                            print(f'Dados ignorados para o arquivo: {nome_arquivo}')


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste.json', 'tabela_teste2.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'processo1': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]},

In [ ]:
# Adicionando a coluna dt_igtao apenas se ela não existir
                                            cursor.execute(f"PRAGMA table_info({nome_tabela})")
                                            columns = [col[1] for col in cursor.fetchall()]
                                            if 'dt_igtao' not in columns:
                                                sql_dt = f'ALTER TABLE {nome_tabela} ADD COLUMN dt_igtao DATE'
                                                cursor.execute(sql_dt)
                                                print('Adicionando campo data ingestão')

In [16]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")
                            
                            for info in tabela_info:
                                banco = info.get("banco", None)
                                nomeArquivoChave = info.get("nomeArquivoChave", None)
                                nome_tabela = info.get("tabela", None)
                                
                                print(f'Nome banco de dados: {banco}')
                                print(f'Nome chave: {nomeArquivoChave}')
                                print(f'Nome tabela: {nome_tabela}')
                                print(f'==============================')
                                
                                with open(ctrl_ing, 'r+') as controle_ingestao:
                                    conteudo_load = controle_ingestao.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave and nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    controle_ingestao.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    controle_ingestao.flush()
                                                    print('=================================')
                                                
                                                if caminho_completo.endswith('.txt'):
                                                    df = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.csv'):
                                                    df = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.bin'):
                                                    pass
                                                else:
                                                    raise ValueError("Formato de arquivo não suportado")
                                                print('Convertendo tipos de dados')
                                                
                                                for col in config_colunas['colunas']:
                                                    if col['tipo'] == 'int':
                                                        df[col['nome']] = df[col['nome']].astype(int)
                                                    elif col['tipo'] == 'float':
                                                        df[col['nome']] = df[col['nome']].astype(float)
                                                    elif col['tipo'] == 'data':
                                                        df[col['nome']] = pd.to_datetime(df[col['nome']], format=col.get('formato', None))
                                                
                                                print('Conectando no banco de dados')
                                                banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                conn = sqlite3.connect(banco_caminho)
                                                print(f'Banco de dados: {banco_caminho}')
                                                print(f'Conectado!')
                                                cursor = conn.cursor()
                                                
                                                print('Criando tabela no banco de dados...')
                                                sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                                for col in config_colunas['colunas']:
                                                    sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                sql_create_table = sql_create_table[:-2] + ")"
                                                cursor.execute(sql_create_table)
                                                
                                                print('Inserindo dados na tabela...')
                                                for index, row in df.iterrows():
                                                    mapped_values = []
                                                    for col in config_colunas['colunas']:
                                                        mapped_values.append(row.get(col['nome'], None))
                                                    print("Valores mapeados:", mapped_values)
                                                    sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                    print("SQL:", sql)
                                                    cursor.execute(sql, mapped_values)
                                                
                                                print('Commit e fechamento da conexão...')
                                                conn.commit()
                                                conn.close()
                                                print("Ingestão de dados concluída com sucesso!")
                                                
                                                print("========================================")
                                                print("Movimentando arquivos")
                                                shutil.move(caminho_completo, caminho_backup)
                                                print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                print(f'Destino: {caminho_backup}')
                                            
                                            else:
                                                print(f'Dados ignorados para o arquivo: {nome_arquivo}')


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste.json', 'tabela_teste2.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {

In [15]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")
                            
                            for info in tabela_info:
                                banco = info.get("banco", banco)
                                nomeArquivoChave = info.get("nomeArquivoChave", None)
                                nome_tabela = info.get("tabela", None)
                                
                                print(f'Nome banco de dados: {banco}')
                                print(f'Nome chave: {nomeArquivoChave}')
                                print(f'Nome tabela: {nome_tabela}')
                                print(f'==============================')
                                
                                with open(ctrl_ing, 'r+') as controle_ingestao:
                                    conteudo_load = controle_ingestao.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave and nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    controle_ingestao.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    controle_ingestao.flush()
                                                    print('=================================')
                                                
                                                if caminho_completo.endswith('.txt'):
                                                    df = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.csv'):
                                                    df = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.bin'):
                                                    pass
                                                else:
                                                    raise ValueError("Formato de arquivo não suportado")
                                                print('Convertendo tipos de dados')
                                                
                                                for col in config_colunas['colunas']:
                                                    if col['tipo'] == 'int':
                                                        df[col['nome']] = df[col['nome']].astype(int)
                                                    elif col['tipo'] == 'float':
                                                        df[col['nome']] = df[col['nome']].astype(float)
                                                    elif col['tipo'] == 'data':
                                                        df[col['nome']] = pd.to_datetime(df[col['nome']], format=col.get('formato', None))
                                                
                                                print('Conectando no banco de dados')
                                                banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                conn = sqlite3.connect(banco_caminho)
                                                print(f'Banco de dados: {banco_caminho}')
                                                print(f'Conectado!')
                                                cursor = conn.cursor()
                                                
                                                print('Criando tabela no banco de dados...')
                                                sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                                for col in config_colunas['colunas']:
                                                    sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                sql_create_table = sql_create_table[:-2] + ")"
                                                cursor.execute(sql_create_table)
                                                
                                                print('Inserindo dados na tabela...')
                                                for index, row in df.iterrows():
                                                    mapped_values = []
                                                    for col in config_colunas['colunas']:
                                                        mapped_values.append(row.get(col['nome'], None))
                                                    print("Valores mapeados:", mapped_values)
                                                    sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                    print("SQL:", sql)
                                                    cursor.execute(sql, mapped_values)
                                                
                                                print('Commit e fechamento da conexão...')
                                                conn.commit()
                                                conn.close()
                                                print("Ingestão de dados concluída com sucesso!")
                                                
                                                print("========================================")
                                                print("Movimentando arquivos")
                                                shutil.move(caminho_completo, caminho_backup)
                                                print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                print(f'Destino: {caminho_backup}')
                                            
                                            else:
                                                print(f'Dados ignorados para o arquivo: {nome_arquivo}')


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste.json', 'tabela_teste2.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {

In [13]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")
                            
                            for info in tabela_info:
                                banco = info.get("banco", banco)
                                nomeArquivoChave = info.get("nomeArquivoChave", None)
                                nome_tabela = info.get("tabela", None)
                                
                                print(f'Nome banco de dados: {banco}')
                                print(f'Nome chave: {nomeArquivoChave}')
                                print(f'Nome tabela: {nome_tabela}')
                                print(f'==============================')
                                
                                with open(ctrl_ing, 'r+') as dados:
                                    conteudo_load = dados.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave and nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    dados.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    dados.flush()
                                                    print('=================================')
                                                
                                                if caminho_completo.endswith('.txt'):
                                                    dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.csv'):
                                                    dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.bin'):
                                                    pass
                                                else:
                                                    raise ValueError("Formato de arquivo não suportado")
                                                print('Convertendo tipos de dados')
                                                
                                                for col in config_colunas['colunas']:
                                                    if col['tipo'] == 'int':
                                                        dados[col['nome']] = dados[col['nome']].astype(int)
                                                    elif col['tipo'] == 'float':
                                                        dados[col['nome']] = dados[col['nome']].astype(float)
                                                    elif col['tipo'] == 'data':
                                                        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))
                                                
                                                print('Conectando no banco de dados')
                                                banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                conn = sqlite3.connect(banco_caminho)
                                                print(f'Banco de dados: {banco_caminho}')
                                                print(f'Conectado!')
                                                cursor = conn.cursor()
                                                
                                                print('Criando tabela no banco de dados...')
                                                sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                                for col in config_colunas['colunas']:
                                                    sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                sql_create_table = sql_create_table[:-2] + ")"
                                                cursor.execute(sql_create_table)
                                                
                                                print('Inserindo dados na tabela...')
                                                for index, row in dados.iterrows():
                                                    mapped_values = []
                                                    for col in config_colunas['colunas']:
                                                        mapped_values.append(row.get(col['nome'], None))
                                                    print("Valores mapeados:", mapped_values)
                                                    sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                    print("SQL:", sql)
                                                    cursor.execute(sql, mapped_values)

                                                sql_dt = f'ALTER TABLE {nome_tabela} ADD COLUMN dt_igtao DATE'
                                                cursor.execute(sql_dt)
                                                print('Adicionando campo data ingestão')
                                                
                                                print('Commit e fechamento da conexão...')
                                                conn.commit()
                                                conn.close()
                                                print("Ingestão de dados concluída com sucesso!")
                                                
                                                print("========================================")
                                                print("Movimentando arquivos")
                                                shutil.move(caminho_completo, caminho_backup)
                                                print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                print(f'Destino: {caminho_backup}')
                                            
                                            else:
                                                print(f'Dados ignorados para o arquivo: {nome_arquivo}')


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste3.json', 'tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {

AttributeError: 'DataFrame' object has no attribute 'write'

In [12]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")
                            
                            for info in tabela_info:
                                banco = info.get("banco", banco)
                                nomeArquivoChave = info.get("nomeArquivoChave", nomeArquivoChave)
                                nome_tabela = info.get("tabela", None)
                                
                                print(f'Nome banco de dados: {banco}')
                                print(f'Nome chave: {nomeArquivoChave}')
                                print(f'Nome tabela: {nome_tabela}')
                                print(f'==============================')
                                
                                with open(ctrl_ing, 'r+') as dados:
                                    conteudo_load = dados.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    dados.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    dados.flush()
                                                    print('=================================')
                                                
                                                if caminho_completo.endswith('.txt'):
                                                    dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.csv'):
                                                    dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                elif caminho_completo.endswith('.bin'):
                                                    pass
                                                else:
                                                    raise ValueError("Formato de arquivo não suportado")
                                                print('Convertendo tipos de dados')
                                                
                                                for col in config_colunas['colunas']:
                                                    if col['tipo'] == 'int':
                                                        dados[col['nome']] = dados[col['nome']].astype(int)
                                                    elif col['tipo'] == 'float':
                                                        dados[col['nome']] = dados[col['nome']].astype(float)
                                                    elif col['tipo'] == 'data':
                                                        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))
                                                
                                                print('Conectando no banco de dados')
                                                banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                conn = sqlite3.connect(banco_caminho)
                                                print(f'Banco de dados: {banco_caminho}')
                                                print(f'Conectado!')
                                                cursor = conn.cursor()
                                                
                                                print('Criando tabela no banco de dados...')
                                                sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                                for col in config_colunas['colunas']:
                                                    sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                sql_create_table = sql_create_table[:-2] + ")"
                                                cursor.execute(sql_create_table)
                                                
                                                print('Inserindo dados na tabela...')
                                                for index, row in dados.iterrows():
                                                    mapped_values = []
                                                    for col in config_colunas['colunas']:
                                                        mapped_values.append(row.get(col['nome'], None))
                                                    print("Valores mapeados:", mapped_values)
                                                    sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                    print("SQL:", sql)
                                                    cursor.execute(sql, mapped_values)

                                                
                                                print('Commit e fechamento da conexão...')
                                                conn.commit()
                                                conn.close()
                                                print("Ingestão de dados concluída com sucesso!")
                                                
                                                print("========================================")
                                                print("Movimentando arquivos")
                                                shutil.move(caminho_completo, caminho_backup)
                                                print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                print(f'Destino: {caminho_backup}')
                                            
                                            else:
                                                print('Dados ignorados!')


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste3.json', 'tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {

In [10]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')


for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)

                for tabela in config_estrutura:
                    if isinstance(tabela, dict):
                        for processo, tabela_info in tabela.items():
                            print(f"Nome do processo: {processo}")
                            
                            for info in tabela_info:
                                    banco = info.get("banco", banco)
                                    nomeArquivoChave = info.get("nomeArquivoChave", nomeArquivoChave)
                                    nome_tabela = info.get("tabela", nome_tabela)
                                    
                                    print(f'Nome banco de dados: {banco}')
                                    print(f'Nome chave: {nomeArquivoChave}')
                                    print(f'Nome tabela: {nome_tabela}')
                                    print(f'==============================')
                                    
                                    with open(ctrl_ing, 'r+') as dados:
                                        conteudo_load = dados.readlines()
                                        print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                    
                                        for root, dirs, lista in os.walk(landing):
                                            print(f'Procurando em {root}...')
                                            for nome_arquivo in lista:
                                                print(f'Verificando arquivo: {nome_arquivo}')
                                                if nomeArquivoChave in nome_arquivo:
                                                    caminho_completo = os.path.join(root, nome_arquivo)
                                                    print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                    sav = os.path.basename(caminho_completo)
                                                    print(f'Conteúdo a gravar: {sav}')
                                                    
                                                    if sav + '\n' not in conteudo_load:
                                                        dados.write(f"{sav}\n")
                                                        print(f'Dado salvo: {sav}')
                                                        dados.flush()
                                                        print('=================================')
                                                    
                                                    if caminho_completo.endswith('.txt'):
                                                        dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.csv'):
                                                        dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.bin'):
                                                        pass
                                                    else:
                                                        raise ValueError("Formato de arquivo não suportado")
                                                    print('Convertendo tipos de dados')
                                                    
                                                    for col in config_colunas['colunas']:
                                                        if col['tipo'] == 'int':
                                                            dados[col['nome']] = dados[col['nome']].astype(int)
                                                        elif col['tipo'] == 'float':
                                                            dados[col['nome']] = dados[col['nome']].astype(float)
                                                        elif col['tipo'] == 'data':
                                                            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))
                                                    
                                                    print('Conectando no banco de dados')
                                                    banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                    conn = sqlite3.connect(banco_caminho)
                                                    print(f'Banco de dados: {banco_caminho}')
                                                    print(f'Conectado!')
                                                    cursor = conn.cursor()
                                                    
                                                    print('Criando tabela no banco de dados...')
                                                    sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
                                                    for col in config_colunas['colunas']:
                                                        sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                    sql_create_table = sql_create_table[:-2] + ")"
                                                    cursor.execute(sql_create_table)
                                                    
                                                    print('Inserindo dados na tabela...')
                                                    for index, row in dados.iterrows():
                                                        mapped_values = []
                                                        for col in config_colunas['colunas']:
                                                            mapped_values.append(row.get(col['nome'], None))
                                                        print("Valores mapeados:", mapped_values)
                                                        sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                        print("SQL:", sql)
                                                        cursor.execute(sql, mapped_values)

                                                

                                                        sql_dt = f'ALTER TABLE {nome_tabela} ADD COLUMN dt_igtao DATE'
                                                        cursor.execute(sql_dt)
                                                        print('Adicionando campo data ingestão')
                                                        
                                                        print('Commit e fechamento da conexão...')
                                                        conn.commit()
                                                        conn.close()
                                                        print("Ingestão de dados concluída com sucesso!")
                                                        
                                                        print("========================================")
                                                        print("Movimentando arquivos")
                                                        shutil.move(caminho_completo, caminho_backup)
                                                        print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                        print(f'Destino: {caminho_backup}')
                                                    
                                                else:
                                                    print('Dados ignorados!')
                                    

Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste_20240621_121201.txt', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste3.json', 'tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {

TypeError: list indices must be integers or slices, not str

Conteúdos do diretório 'landing': ['archiving_ctrl']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}


AttributeError: 'str' object has no attribute 'get'

In [77]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

try:
    for item in read_layouts:
        accept = os.path.join(layouts, item)
        if os.path.isdir(accept):
            print(f'{item} é uma pasta')
            subconteudos = os.listdir(accept)
            print(f'Conteúdos de {item}: {subconteudos}')
            
            caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
            if os.path.exists(caminho_estrutura_json):
                print(f'Encontrado estrutura.json em {item}')
                
                caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
                print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
                
                with open(caminho_tabela_json, 'r') as f:
                    config_colunas = json.load(f)
                    print('Lendo estrutura de coluna json')
                
                with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)

                    for tabela in config_estrutura:
                        if isinstance(tabela, dict):
                            for tabela_nome, tabela_info in tabela.items():
                                print(f"Nome da tabela: {tabela_nome}")
                                banco = None
                                nomeArquivoChave = None
                                nome_tabela = None
                                
                                for info in tabela_info:
                                    banco = info.get("banco", banco)
                                    nomeArquivoChave = info.get("nomeArquivoChave", nomeArquivoChave)
                                    nome_tabela = info.get("tabela", nome_tabela)
                                
                                print(f"Banco: {banco}")
                                print(f"Nome Arquivo Chave: {nomeArquivoChave}")
                                print(f"Nome da Tabela: {nome_tabela}")
                                print('=====================')
                                
                                with open(ctrl_ing, 'r+') as dados:
                                    conteudo_load = dados.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                    
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    dados.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    dados.flush()
                                                    print('=================================')
                                                    
                                                    if caminho_completo.endswith('.txt'):
                                                        dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.csv'):
                                                        dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.bin'):
                                                        pass
                                                    else:
                                                        raise ValueError("Formato de arquivo não suportado")
                                                    print('Convertendo tipos de dados')
                                                    
                                                    for col in config_colunas['colunas']:
                                                        if col['tipo'] == 'int':
                                                            dados[col['nome']] = dados[col['nome']].astype(int)
                                                        elif col['tipo'] == 'float':
                                                            dados[col['nome']] = dados[col['nome']].astype(float)
                                                        elif col['tipo'] == 'data':
                                                            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))
                                                    
                                                    print('Conectando no banco de dados')
                                                    banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                    conn = sqlite3.connect(banco_caminho)
                                                    print(f'Banco de dados: {banco_caminho}')
                                                    print(f'Conectado!')
                                                    cursor = conn.cursor()

                                                    print('Criando tabela no banco de dados...')
                                                    sql_create_table = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ("
                                                    for col in config_colunas['colunas']:
                                                        sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                    sql_create_table = sql_create_table[:-2] + ")"
                                                    cursor.execute(sql_create_table)

                                                    print('Inserindo dados na tabela...')
                                                    for index, row in dados.iterrows():
                                                        mapped_values = []
                                                        for col in config_colunas['colunas']:
                                                            mapped_values.append(row.get(col['nome'], None))
                                                        print("Valores mapeados:", mapped_values)
                                                        sql = f"INSERT INTO {nome_tabela} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                        print("SQL:", sql)
                                                        cursor.execute(sql, mapped_values)

                                                    sql_dt = f'ALTER TABLE {nome_tabela} ADD COLUMN dt_igtao DATE'
                                                    cursor.execute(sql_dt)
                                                    print('Adicionando campo data ingestão')
                                                    
                                                    print('Commit e fechamento da conexão...')
                                                    conn.commit()
                                                    conn.close()
                                                    print("Ingestão de dados concluída com sucesso!")
                                                    
                                                    print("========================================")
                                                    print("Movimentando arquivos")
                                                    shutil.move(caminho_completo, caminho_backup)
                                                    print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                    print(f'Destino: {caminho_backup}')
                                                    
                                                else:
                                                    print('Dados ignorados!')
except Exception as error:
    print(error)


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste3.json', 'tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {'teste02': [{'banco': 'banco_teste_novo

In [78]:
import sqlite3

# Caminho para o banco de dados SQLite
database_path = '/workspaces/api-geral/ING/database/banco_teste_novo.db'

# Conectando ao banco de dados
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Executando a consulta para listar as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recuperando e imprimindo o resultado
tables = cursor.fetchall()

print("Tabelas no banco de dados:")
for table in tables:
    print(table[0])

# Fechando a conexão
conn.close()


Tabelas no banco de dados:
teste01
tabela_teste


In [68]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

try:
    for item in read_layouts:
        accept = os.path.join(layouts, item)
        if os.path.isdir(accept):
            print(f'{item} é uma pasta')
            subconteudos = os.listdir(accept)
            print(f'Conteúdos de {item}: {subconteudos}')
            
            caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
            if os.path.exists(caminho_estrutura_json):
                print(f'Encontrado estrutura.json em {item}')
                
                caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
                print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
                
                with open(caminho_tabela_json, 'r') as f:
                    config_colunas = json.load(f)
                    print('Lendo estrutura de coluna json')
                
                with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)

                    for tabela in config_estrutura:
                        if isinstance(tabela, dict):
                            for tabela_nome, tabela_info in tabela.items():
                                print(f"Nome da tabela: {tabela_nome}")
                                banco = None
                                nomeArquivoChave = None
                                for info in tabela_info:
                                    banco = info.get("banco", banco)
                                    nomeArquivoChave = info.get("nomeArquivoChave", nomeArquivoChave)
                                
                                print(f"Banco: {banco}")
                                print(f"Nome Arquivo Chave: {nomeArquivoChave}")
                                print('=====================')
                                
                                with open(ctrl_ing, 'r+') as dados:
                                    conteudo_load = dados.readlines()
                                    print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                                    
                                    for root, dirs, lista in os.walk(landing):
                                        print(f'Procurando em {root}...')
                                        for nome_arquivo in lista:
                                            print(f'Verificando arquivo: {nome_arquivo}')
                                            if nomeArquivoChave in nome_arquivo:
                                                caminho_completo = os.path.join(root, nome_arquivo)
                                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                                sav = os.path.basename(caminho_completo)
                                                print(f'Conteúdo a gravar: {sav}')
                                                
                                                if sav + '\n' not in conteudo_load:
                                                    dados.write(f"{sav}\n")
                                                    print(f'Dado salvo: {sav}')
                                                    dados.flush()
                                                    print('=================================')
                                                    
                                                    if caminho_completo.endswith('.txt'):
                                                        dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.csv'):
                                                        dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                                    elif caminho_completo.endswith('.bin'):
                                                        pass
                                                    else:
                                                        raise ValueError("Formato de arquivo não suportado")
                                                    print('Convertendo tipos de dados')
                                                    
                                                    for col in config_colunas['colunas']:
                                                        if col['tipo'] == 'int':
                                                            dados[col['nome']] = dados[col['nome']].astype(int)
                                                        elif col['tipo'] == 'float':
                                                            dados[col['nome']] = dados[col['nome']].astype(float)
                                                        elif col['tipo'] == 'data':
                                                            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))
                                                    
                                                    print('Conectando no banco de dados')
                                                    banco_caminho = f'/workspaces/api-geral/ING/database/{banco}'
                                                    conn = sqlite3.connect(banco_caminho)
                                                    print(f'Banco de dados: {banco_caminho}')
                                                    print(f'Conectado!')
                                                    cursor = conn.cursor()

                                                    print('Criando tabela no banco de dados...')
                                                    sql_create_table = f"CREATE TABLE IF NOT EXISTS {tabela_nome} ("
                                                    for col in config_colunas['colunas']:
                                                        sql_create_table += f"{col['nome']} {col['tipo']}, "
                                                    sql_create_table = sql_create_table[:-2] + ")"
                                                    cursor.execute(sql_create_table)

                                                    print('Inserindo dados na tabela...')
                                                    for index, row in dados.iterrows():
                                                        mapped_values = []
                                                        for col in config_colunas['colunas']:
                                                            mapped_values.append(row.get(col['nome'], None))
                                                        print("Valores mapeados:", mapped_values)
                                                        sql = f"INSERT INTO {tabela_nome} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                        print("SQL:", sql)
                                                        cursor.execute(sql, mapped_values)

                                                    sql_dt = f'ALTER TABLE {tabela_nome} ADD COLUMN dt_igtao DATE'
                                                    cursor.execute(sql_dt)
                                                    print('Adicionando campo data ingestão')
                                                    
                                                    print('Commit e fechamento da conexão...')
                                                    conn.commit()
                                                    conn.close()
                                                    print("Ingestão de dados concluída com sucesso!")
                                                    
                                                    print("========================================")
                                                    print("Movimentando arquivos")
                                                    shutil.move(caminho_completo, caminho_backup)
                                                    print(f'Arquivo enviado para bckp: {caminho_completo}')
                                                    print(f'Destino: {caminho_backup}')
                                                    
                                                else:
                                                    print('Dados ignorados!')
except Exception as error:
    print(error)


Conteúdos do diretório 'landing': ['archiving_ctrl', 'resultado_teste2_20240621_101010.txt']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
teste é uma pasta
Conteúdos de teste: ['tabela_teste3.json', 'tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json
Lendo estrutura de coluna json
Configuração estrutura: [{'teste01': [{'banco': 'banco_teste_novo.db'}, {'tabela': 'tabela_teste'}, {'descricao': 'tabela teste'}, {'nomeArquivoChave': 'teste'}]}, {'teste02': [{'banco': 'banco_teste_novo

In [75]:
import sqlite3

# Caminho para o banco de dados SQLite
database_path = '/workspaces/api-geral/ING/database/banco_teste_novo.db'

# Conectando ao banco de dados
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Executando a consulta para listar as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recuperando e imprimindo o resultado
tables = cursor.fetchall()

print("Tabelas no banco de dados:")
for table in tables:
    print(table[0])

# Fechando a conexão
conn.close()


Tabelas no banco de dados:
teste01


In [76]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_teste_novo.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'teste01'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)  

Resultado:
(10, 'nene', None)
(11, 'roseli', None)
(12, 'juliano', None)


In [ ]:
gravar = []
for infos in transaction:
    gravar.append((infos['object'], infos['status']))

In [67]:
import os
import json

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(caminho_tabela_json, 'r') as f:
                config_colunas = json.load(f)
                print('Lendo estrutura de coluna json')
                print(config_colunas)
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)
                
                for tabela in config_estrutura:
                    if isinstance(tabela, dict):  # Verificar se tabela é um dicionário
                        for tabela_nome, tabela_info in tabela.items():
                            print(f"Nome da tabela: {tabela_nome}")
                            banco = None
                            tabela_nome_val = None
                            descricao = None
                            nomeArquivoChave = None
                            for info in tabela_info:
                                banco = info.get("banco", banco)
                                tabela_nome_val = info.get("tabela", tabela_nome_val)
                                descricao = info.get("descricao", descricao)
                                nomeArquivoChave = info.get("nomeArquivoChave", nomeArquivoChave)
                            print(f"Banco: {banco}")
                            print(f"Tabela: {tabela_nome_val}")
                            print(f"Descrição: {descricao}")
                            print(f"Nome Arquivo Chave: {nomeArquivoChave}")
                            print('=====================')


Conteúdos do diretório 'landing': ['resultado_teste3_20240621_101010.txt', 'archiving_ctrl']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
Lendo estrutura de coluna json
{'colunas': [{'nome': 'id', 'tipo': 'string'}, {'nome': 'nome', 'tipo': 'string'}, {'nome': 'id_mun', 'tipo': 'string'}, {'nome': 'nm_mun', 'tipo': 'string'}, {'nome': 'id_mcr', 'tipo': 'string'}, {'nome': 'nm_mcr', 'tipo': 'string'}, {'nome': 'id_uf', 'tipo': 'string'}, {'nome': 'nm_uf', 'tipo': 'string'}, {'nome': 'sigla_uf', 'tipo': 'sigla_uf'}, {'nome': 'id_reg', 'tipo': 'string'}, {'nome': 'nm_reg', 'tipo': 'string'}, {'nome': 'sigla_rg', 'tipo': 'string'}, {'nome': 'dtigtao', 'tipo': 'string'}]}
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste

In [63]:
caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'


read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')


for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'Conteúdos de {item}: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
            print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
            
            with open(f'{caminho_tabela_json}', 'r') as f:
                config_colunas = json.load(f)
                print('lendo estrutura de coluna json')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print("Configuração estrutura:", config_estrutura)
                for tabela in config_estrutura:
                    for tabela_nome, tabela_info in tabela.items():
                        print(f"Nome da tabela: {tabela_nome}")
                        for info in tabela_info:
                            banco = info.get("banco")
                            tabela = info.get("tabela")
                            descricao = info.get("descricao")
                            nomeArquivoChave = info.get("nomeArquivoChave")
                            print(f"Banco: {banco}")
                            print(f"Tabela: {tabela}")
                            print(f"Descrição: {descricao}")
                            print(f"Nome Arquivo Chave: {nomeArquivoChave}")
                            print('=====================')
                
                # key = config_estrutura['nomeArquivoChave'][0]
                # database = config_estrutura(['banco'][0])
                    # print(f'Palavra chave: {key}')
                    # print('=====================')

Conteúdos do diretório 'landing': ['resultado_teste3_20240621_101010.txt', 'archiving_ctrl']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}


AttributeError: 'str' object has no attribute 'items'

In [20]:
data_igtao = datetime.now().strftime('%Y-%m-%d')
teste = f'teste_{data_igtao}.txt'
print(teste)
print(data_igtao)

teste_2024-06-21.txt
2024-06-21


In [23]:
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')
ultimo_timestamp = None

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

try:
    for item in read_layouts:
        accept = os.path.join(layouts, item)
        if os.path.isdir(accept):
            print(f'{item} é uma pasta')
            subconteudos = os.listdir(accept)
            print(f'Conteúdos de {item}: {subconteudos}')
            
            caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
            if os.path.exists(caminho_estrutura_json):
                print(f'Encontrado estrutura.json em {item}')
                
                caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
                print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
                
                with open(f'{caminho_tabela_json}', 'r') as f:
                    config_colunas = json.load(f)
                    print('lendo estrutura de coluna json')
                
                with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)
                    key = config_estrutura.get('nomeArquivoChave')
                    database = config_estrutura.get('banco')
                    print(f'Palavra chave: {key}')
                    print('=====================')
                
                with open(ctrl_ing, 'r+') as dados:
                    conteudo_load = dados.readlines()
                    print(f'Conteúdo do arquivo de backup: {conteudo_load}')
                    
                    for root, dirs, lista in os.walk(landing):
                        print(f'Procurando em {root}...')
                        for nome_arquivo in lista:
                            print(f'Verificando arquivo: {nome_arquivo}')
                            if key in nome_arquivo:
                                caminho_completo = os.path.join(root, nome_arquivo)
                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                sav = os.path.basename(caminho_completo)
                                print(f'Conteúdo a gravar: {sav}')
                                    
                                if sav + '\n' not in conteudo_load:
                                    dados.write(f"{sav}\n")
                                    print(f'Dado salvo: {sav}')
                                    dados.flush()
                                    print('=================================')
                                    
                                    if caminho_completo.endswith('.txt'):
                                        dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                    elif caminho_completo.endswith('.csv'):
                                        dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                    elif caminho_completo.endswith('.bin'):
                                        pass
                                    else:
                                        raise ValueError("Formato de arquivo não suportado")
                                    print('convertendo tipos de dados')

                                    for col in config_colunas['colunas']:
                                        if col['tipo'] == 'int':
                                            dados[col['nome']] = dados[col['nome']].astype(int)
                                        elif col['tipo'] == 'float':
                                            dados[col['nome']] = dados[col['nome']].astype(float)
                                        elif col['tipo'] == 'data':
                                            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

                                    print('conctando no banco de dados')
                                    banco = f'/workspaces/api-geral/ING/database/{database}'
                                    conn = sqlite3.connect(f'{banco}')
                                    print(f'banco de dados: {database}')
                                    print(f'conectado!')
                                    cursor = conn.cursor()

                                    print('Criando tabela no banco de dados...')
                                    sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
                                    for col in config_colunas['colunas']:
                                        sql_create_table += f"{col['nome']} {col['tipo']}, "
                                    sql_create_table = sql_create_table[:-2] + ")"
                                    cursor.execute(sql_create_table)

                                    print('Inserindo dados na tabela...')
                                    for index, row in dados.iterrows():
                                        mapped_values = []
                                        for col in config_colunas['colunas']:
                                            mapped_values.append(row.get(col['nome'], None))
                                        print("Valores mapeados:", mapped_values)
                                        sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                        print("SQL:", sql)
                                        sql_dt = f'ALTER TABLE {data_igtao} ADD COLUMN dt_igtao DATE'
                                        cursor.execute(sql, mapped_values)
                                        cursor.execute(sql_dt)
                                        print('adicionando campo data ingestao')
                                        

                                    print('Commit e fechamento da conexão...')
                                    conn.commit()
                                    conn.close()
                                    print("Ingestão de dados concluída com sucesso!")
                                    
                                    print("========================================")
                                    print("movimentando arquivos")
                                    shutil.move(caminho_completo, caminho_backup)
                                    print(f'arquivo enviado para bckp: {caminho_completo}')
                                    print(f'destino: {caminho_backup}')
                                    
                                    
                                else:
                                    print('dados ignorado!')
except FileExistsError as error:
    print(error)

Conteúdos do diretório 'landing': ['resultado_teste3_20240621_101010.txt', 'archiving_ctrl']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
Palavra chave: geo
Conteúdo do arquivo de backup: ['resultado_teste_20240620_204501.txt\n', 'resultado_teste_20240620_205621.txt\n', 'resultado_concat_20240513_162447.txt\n', 'resultado_teste_20240617_222901.txt\n']
Procurando em /workspaces/api-geral/ING/app/src/data/landing/...
Verificando arquivo: resultado_teste3_20240621_101010.txt
Procurando em /workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl...
Verificando arquivo: controle_ingestao.txt
teste é uma 

AttributeError: 'list' object has no attribute 'get'

In [16]:
import os
import json
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
ultimo_timestamp = None

caminho_backup = '/workspaces/api-geral/ING/app/src/bckp/'
ctrl_ing = "/workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl/controle_ingestao.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('=====================')

try:
    for item in read_layouts:
        accept = os.path.join(layouts, item)
        if os.path.isdir(accept):
            print(f'{item} é uma pasta')
            subconteudos = os.listdir(accept)
            print(f'Conteúdos de {item}: {subconteudos}')
            
            caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
            if os.path.exists(caminho_estrutura_json):
                print(f'Encontrado estrutura.json em {item}')
                
                caminho_tabela_json = os.path.join(accept, f'tabela_{item}.json')
                print(f'Encontrando schema da tabela.json em {caminho_tabela_json}')
                
                with open(f'{caminho_tabela_json}', 'r') as f:
                    config_colunas = json.load(f)
                    print('lendo estrutura de coluna json')
                
                with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)
                    key = config_estrutura.get('nomeArquivoChave')
                    database = config_estrutura.get('banco')
                    print(f'Palavra chave: {key}')
                    print('=====================')
                
                with open(ctrl_ing, 'r+') as dados:
                    conteudo_load = dados.readlines()
                    print(f'Conteúdo do arquivo de backup: {conteudo_load}')
                    
                    for root, dirs, lista in os.walk(landing):
                        print(f'Procurando em {root}...')
                        for nome_arquivo in lista:
                            print(f'Verificando arquivo: {nome_arquivo}')
                            if key in nome_arquivo:
                                caminho_completo = os.path.join(root, nome_arquivo)
                                print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                sav = os.path.basename(caminho_completo)
                                print(f'Conteúdo a gravar: {sav}')
                                    
                                if sav + '\n' not in conteudo_load:
                                    dados.write(f"{sav}\n")
                                    print(f'Dado salvo: {sav}')
                                    dados.flush()
                                    print('=================================')
                                    
                                    if caminho_completo.endswith('.txt'):
                                        dados = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                    elif caminho_completo.endswith('.csv'):
                                        dados = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas']])
                                    elif caminho_completo.endswith('.bin'):
                                        pass
                                    else:
                                        raise ValueError("Formato de arquivo não suportado")
                                    print('convertendo tipos de dados')

                                    for col in config_colunas['colunas']:
                                        if col['tipo'] == 'int':
                                            dados[col['nome']] = dados[col['nome']].astype(int)
                                        elif col['tipo'] == 'float':
                                            dados[col['nome']] = dados[col['nome']].astype(float)
                                        elif col['tipo'] == 'data':
                                            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

                                    print('conctando no banco de dados')
                                    banco = f'/workspaces/api-geral/ING/database/{database}'
                                    conn = sqlite3.connect(f'{banco}')
                                    print(f'banco de dados: {database}')
                                    print(f'conectado!')
                                    cursor = conn.cursor()

                                    print('Criando tabela no banco de dados...')
                                    sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
                                    for col in config_colunas['colunas']:
                                        sql_create_table += f"{col['nome']} {col['tipo']}, "
                                    sql_create_table = sql_create_table[:-2] + ")"
                                    cursor.execute(sql_create_table)

                                    print('Inserindo dados na tabela...')
                                    for index, row in dados.iterrows():
                                        mapped_values = []
                                        for col in config_colunas['colunas']:
                                            mapped_values.append(row.get(col['nome'], None))
                                        print("Valores mapeados:", mapped_values)
                                        sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                        print("SQL:", sql)
                                        cursor.execute(sql, mapped_values)

                                    print('Commit e fechamento da conexão...')
                                    conn.commit()
                                    conn.close()
                                    shutil.move(caminho_completo, caminho_backup)
                                    # os.rename(caminho_completo, caminho_backup)
                                    print(f'arquivo enviado para bckp: {caminho_completo}')
                                    print(f'destino: {caminho_backup}')
                                    print("Ingestão de dados concluída com sucesso!")
                                    
                                else:
                                    print('dados ignorado!')
except FileExistsError as error:
    print(error)


Conteúdos do diretório 'landing': ['resultado_teste_20240617_222901.txt', 'archiving_ctrl']
Conteúdos do diretório 'layouts': ['geo', 'teste', 'cep']
geo é uma pasta
Conteúdos de geo: ['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
Encontrando schema da tabela.json em /workspaces/api-geral/ING/app/src/layouts/geo/tabela_geo.json
lendo estrutura de coluna json
Configuração estrutura: {'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
Palavra chave: geo
Conteúdo do arquivo de backup: ['resultado_teste_20240620_204501.txt\n', 'resultado_teste_20240620_205621.txt\n', 'resultado_concat_20240513_162447.txt\n']
Procurando em /workspaces/api-geral/ING/app/src/data/landing/...
Verificando arquivo: resultado_teste_20240617_222901.txt
Procurando em /workspaces/api-geral/ING/app/src/data/landing/archiving_ctrl...
Verificando arquivo: controle_ingestao.txt
teste é uma pasta
Conteúdos de teste: ['tabela_teste.js

In [9]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'tabela_teste'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)    

Resultado:
('id', 'nome')
(1, 'teste')
(2, 'teste2')
(3, 'teste3')
('id', 'nome')
(4, 'rafael')
(5, 'joyce')
(6, 'phoebe')
(7, 'val')
(8, 'douglas')
(9, 'wesley')


In [ ]:
caminho_origem = '/caminho/do/arquivo.txt'
caminho_destino = '/workspaces/api-geral/ING/app/src/bckp/'

os.rename(caminho_origem, caminho_destino)

In [ ]:
import json
import pandas as pd
import sqlite3
import os
from datetime import datetime
import os.path
import time
import sys


arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/backup_arquivo.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/cep/estrutura.json'
file_col_json = '/workspaces/api-geral/ING/app/src/layouts/cep/tabela_cep.json'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

with open(file_tb_json, 'r') as f:
    config_estrutura = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_estrutura)
    key = config_estrutura['nomeArquivoChave']
    database = config_estrutura['banco']
    print(f"palavra chave: {key}")

try:  
    with open(arq_dir, 'r+') as dados:
        conteudo_load = dados.readlines()
        print(f'dados conteudo_load: {conteudo_load}')

        for root, dirs, lista in os.walk(landing):
            for nome_arquivo in lista:
                if key in nome_arquivo:
                    caminho_completo = os.path.join(root, nome_arquivo)
                    print(f'Arquivo encontrado: {caminho_completo}')
                    sav = os.path.basename(caminho_completo)
                    print(f'conteudo a gravar: {sav}')
                    
                    if sav + '\n' not in conteudo_load:
                        dados.write(f"{sav}\n")
                        print(f'dados salvo: {sav}')
                        print('dados gravado!')
                        dados.flush()
                        os.ftruncate(dados.fileno(), dados.tell())
                        print('=================================')
                        
                        if caminho_completo.endswith('.txt'):
                            dados = pd.read_csv(caminho_completo, delimiter=';', header=None, names=[col['nome'] for col in config_colunas['colunas']])
                        elif caminho_completo.endswith('.csv'):
                            dados = pd.read_csv(caminho_completo, header=None, names=[col['nome'] for col in config_colunas['colunas']])
                        elif caminho_completo.endswith('.bin'):
                            pass
                        else:
                            raise ValueError("Formato de arquivo não suportado")

                        print('convertendo tipos de dados')

                        for col in config_colunas['colunas']:
                            if col['tipo'] == 'int':
                                dados[col['nome']] = dados[col['nome']].astype(int)
                            elif col['tipo'] == 'float':
                                dados[col['nome']] = dados[col['nome']].astype(float)
                            elif col['tipo'] == 'data':
                                dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

                        print('conctando no banco de dados')
                        banco = f'/workspaces/api-geral/ING/database/{database}'
                        conn = sqlite3.connect(f'{banco}')
                        print(f'banco de dados: {database}')
                        print(f'conectado!')
                        cursor = conn.cursor()

                        print('Criando tabela no banco de dados...')
                        sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
                        for col in config_colunas['colunas']:
                            sql_create_table += f"{col['nome']} {col['tipo']}, "
                        sql_create_table = sql_create_table[:-2] + ")"
                        cursor.execute(sql_create_table)

                        print('Inserindo dados na tabela...')
                        for index, row in dados.iterrows():
                            mapped_values = []
                            for col in config_colunas['colunas']:
                                mapped_values.append(row.get(col['nome'], None))
                            print("Valores mapeados:", mapped_values)
                            sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                            print("SQL:", sql)
                            cursor.execute(sql, mapped_values)

                        print('Commit e fechamento da conexão...')
                        conn.commit()
                        conn.close()

                        print("Ingestão de dados concluída com sucesso!")
                    else:
                        print('dados ignorado!')
except FileExistsError as error:
    print(error)

In [41]:
import os
import re
import json
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
ultimo_timestamp = None
caminho_arquivo = None

backup_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
layouts = '/workspaces/api-geral/ING/app/src/layouts/'

read_landing = os.listdir(landing)
print(read_landing)
read_layouts = os.listdir(layouts)
print(read_layouts)
print('=====================')

for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print(f'{item} é uma pasta')
        subconteudos = os.listdir(accept)
        print(f'{subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            with open(caminho_estrutura_json, 'r') as f:
                config_estrutura = json.load(f)
                print(config_estrutura)
                key = config_estrutura['nomeArquivoChave']
                print(f'palavra chave: {key}')
                print('=====================')
            
                with open(backup_dir, 'r+') as dados:
                    conteudo_load = dados.readlines()
                    print(f'dados conteudo_load: {conteudo_load}')

                for root, dirs, lista in os.walk(landing):
                    for nome_arquivo in lista:
                        if key in nome_arquivo:
                            caminho_completo = os.path.join(root, nome_arquivo)
                            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                                ultimo_timestamp = timestamp
                                print(f'Arquivo encontrado: {caminho_completo}')
                                sav = os.path.basename(caminho_completo)
                                print(f'conteudo a gravar: {sav}')
                                
                                if sav + '\n' not in conteudo_load:
                                    for save in dados:
                                        dados.writelines(f"{save}\n")
                                        print(f'dados salvo: {save}')
                                        print('dados gravado!')
                                        dados.flush()
                                        
os.ftruncate(dados.fileno(), dados.tell())
print('=================================')
  

['resultado_teste_20240617_222901.txt', 'resultado_concat_20240513_162447.txt']
['geo', 'teste', 'cep']
geo é uma pasta
['tabela_geo.json', 'estrutura.json']
Encontrado estrutura.json em geo
{'banco': 'banco_cep_teste.db', 'tabela': 'tb_geo_brnz', 'descricao': 'tabela teste geo', 'nomeArquivoChave': 'geo'}
palavra chave: geo
dados conteudo_load: []
teste é uma pasta
['tabela_teste.json', 'estrutura.json']
Encontrado estrutura.json em teste
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste', 'nomeArquivoChave': 'teste'}
palavra chave: teste
dados conteudo_load: []
Arquivo encontrado: /workspaces/api-geral/ING/app/src/data/landing/resultado_teste_20240617_222901.txt
conteudo a gravar: resultado_teste_20240617_222901.txt


ValueError: I/O operation on closed file.

In [4]:
arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'
file_col_json = '/workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

with open(file_tb_json, 'r') as f:
    config_estrutura = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_estrutura)
    key = config_estrutura['nomeArquivoChave']
    database = config_estrutura['banco']
    print(f"palavra chave: {key}")

try:  
    with open(arq_dir, 'r+') as dados:
        conteudo_load = dados.readlines()
        print(f'dados conteudo_load: {conteudo_load}')

        for root, dirs, lista in os.walk(landing):
            for nome_arquivo in lista:
                if key in nome_arquivo:
                    caminho_completo = os.path.join(root, nome_arquivo)
                    if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                        ultimo_timestamp = timestamp
                        print(f'Arquivo encontrado: {caminho_completo}')
                        sav = os.path.basename(caminho_completo)
                        print(f'conteudo a gravar: {sav}')
                        
                        if sav + '\n' not in conteudo_load:
                            dados.write(f"{sav}\n")
                            print(f'dados salvo: {sav}')
                            print('dados gravado!')
                            dados.flush()
                            os.ftruncate(dados.fileno(), dados.tell())
                            print('=================================')
                            
                            if caminho_completo.endswith('.txt'):
                                dados = pd.read_csv(caminho_completo, delimiter=';', header=None, names=[col['nome'] for col in config_colunas['colunas']])
                            elif caminho_completo.endswith('.csv'):
                                dados = pd.read_csv(caminho_completo, header=None, names=[col['nome'] for col in config_colunas['colunas']])
                            elif caminho_completo.endswith('.bin'):
                                pass
                            else:
                                raise ValueError("Formato de arquivo não suportado")

                            print('convertendo tipos de dados')

                            for col in config_colunas['colunas']:
                                if col['tipo'] == 'int':
                                    dados[col['nome']] = dados[col['nome']].astype(int)
                                elif col['tipo'] == 'float':
                                    dados[col['nome']] = dados[col['nome']].astype(float)
                                elif col['tipo'] == 'data':
                                    dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

                            print('conctando no banco de dados')
                            banco = f'/workspaces/api-geral/ING/database/{database}'
                            conn = sqlite3.connect(f'{banco}')
                            print(f'banco de dados: {database}')
                            print(f'conectado!')
                            cursor = conn.cursor()

                            print('Criando tabela no banco de dados...')
                            sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
                            for col in config_colunas['colunas']:
                                sql_create_table += f"{col['nome']} {col['tipo']}, "
                            sql_create_table = sql_create_table[:-2] + ")"
                            cursor.execute(sql_create_table)

                            print('Inserindo dados na tabela...')
                            for index, row in dados.iterrows():
                                mapped_values = []
                                for col in config_colunas['colunas']:
                                    mapped_values.append(row.get(col['nome'], None))
                                print("Valores mapeados:", mapped_values)
                                sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                print("SQL:", sql)
                                cursor.execute(sql, mapped_values)

                            print('Commit e fechamento da conexão...')
                            conn.commit()
                            conn.close()

                            print("Ingestão de dados concluída com sucesso!")
                        else:
                            print('dados ignorado!')
except FileExistsError as error:
    print(error)

lendo estrutura de coluna json
lendo estrutura de coluna json
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste', 'nomeArquivoChave': 'teste'}
palavra chave: teste
dados conteudo_load: ['\n']
Arquivo encontrado: /workspaces/api-geral/ING/app/src/data/landing/resultado_teste_20240617_222901.txt
conteudo a gravar: resultado_teste_20240617_222901.txt
dados salvo: resultado_teste_20240617_222901.txt
dados gravado!
convertendo tipos de dados
conctando no banco de dados
banco de dados: banco_teste.db
conectado!
Criando tabela no banco de dados...
Inserindo dados na tabela...
Valores mapeados: ['id', 'nome']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['1', 'teste']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['2', 'teste2']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['3', 'teste3']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Commit e fechamento da conexão...
Inges

In [5]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'tabela_teste'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)    

Resultado:
('id', 'nome')
(1, 'teste')
(2, 'teste2')
(3, 'teste3')


In [ ]:
              
        # with open(subconteudos, 'r') as f:
        # config_estrutura = json.load(f)
        # # print('lendo estrutura de coluna json')
        # # print(config_estrutura)
        # key = config_estrutura['nomeArquivoChave']







# # Caminho do diretório principal
# diretorio_principal = '/caminho/do/diretorio'

# # Lista os conteúdos do diretório principal
# conteudos = os.listdir(diretorio_principal)

# # Itera sobre os conteúdos do diretório
# for item in conteudos:
#     # Cria o caminho completo do item
#     caminho_completo = os.path.join(diretorio_principal, item)
    
#     # Verifica se o item é uma pasta
#     if os.path.isdir(caminho_completo):
#         print(f'{item} é uma pasta')
        
#         # Acesse o conteúdo da pasta
#         subconteudos = os.listdir(caminho_completo)
#         print(f'Conteúdos de {item}: {subconteudos}')
        
#         # Aqui você pode adicionar qualquer lógica adicional para manipular os conteúdos da pasta
#     else:
#         print(f'{item} não é uma pasta')

# # Exemplo adicional: acessando o conteúdo de uma pasta específica
# pasta_especifica = 'nome_da_pasta'
# caminho_pasta_especifica = os.path.join(diretorio_principal, pasta_especifica)

# if os.path.isdir(caminho_pasta_especifica):
#     conteudos_pasta_especifica = os.listdir(caminho_pasta_especifica)
#     print(f'Conteúdos de {pasta_especifica}: {conteudos_pasta_especifica}')
# else:
#     print(f'{pasta_especifica} não é uma pasta válida')





# for i in read_landing: 
#     regex = re.search(r'_(.*?)_', str(i))
#     print(regex)
#     novo_arq.append(regex)
#     print(f'arquivo append: {novo_arq}')
    
# for search in landing:
#     with open(file_tb_json, 'r') as f:
#         config_estrutura = json.load(f)
#         # print('lendo estrutura de coluna json')
#         # print(config_estrutura)
#         key = config_estrutura['nomeArquivoChave']
#         for root, dirs, lista in os.walk(landing):
#             for nome_arquivo in lista:
#                 if key in nome_arquivo:
#                     caminho_completo = os.path.join(root, nome_arquivo)
#                     print(caminho_completo)
        # print(key)
        
    
    # for root, dirs, lista in os.walk(landing):
    #         for nome_arquivo in lista:
    #             if regex in nome_arquivo:
    #                 caminho_completo = os.path.join(root, nome_arquivo)
    #                 print(caminho_completo)
    
    # regex = os.replace('_', '')
    # print(regex)
# regex = re.search(r'_(.?)_', read_landing)
# print(regex)
# m=re.search(r'<(.*?)>','abevbv envvrhwkv <eiwbv> ebvi <wieunv> ajhbsvhj')

In [101]:
banco = f'/workspaces/api-geral/ING/database/banco_teste.db'
conn = sqlite3.connect(f'{banco}')
print(f'banco de dados: {database}')
conn.cursor()
query = f'''
drop database {banco}
'''
cursor.execute(query)
conn.commit()
conn.close()

banco de dados: banco_teste.db


ProgrammingError: Cannot operate on a closed database.

In [96]:
arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'
file_col_json = '/workspaces/api-geral/ING/app/src/layouts/teste/tabela_teste.json'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

with open(file_tb_json, 'r') as f:
    config_estrutura = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_estrutura)
    key = config_estrutura['nomeArquivoChave']
    print(f"palavra chave: {key}")

try:  
    with open(arq_dir, 'r+') as dados:
        conteudo_load = dados.readlines()
        print(f'dados conteudo_load: {conteudo_load}')

        for root, dirs, lista in os.walk(landing):
            for nome_arquivo in lista:
                if key in nome_arquivo:
                    caminho_completo = os.path.join(root, nome_arquivo)
                    print(f'Arquivo encontrado: {caminho_completo}')
                    sav = os.path.basename(caminho_completo)
                    
                    print(f'conteudo a gravar: {sav}')
                    
                    if sav + '\n' not in conteudo_load:
                        dados.write(f"{sav}\n")
                        print(f'dados salvo: {sav}')
                        print('dados gravado!')
                    else:
                        print('dados ignorado!')
        dados.flush()
        os.ftruncate(dados.fileno(), dados.tell())


    if caminho_completo.endswith('.txt'):
        dados = pd.read_csv(caminho_completo, delimiter=';', header=None, names=[col['nome'] for col in config_colunas['colunas']])
    elif caminho_completo.endswith('.csv'):
        dados = pd.read_csv(caminho_completo, header=None, names=[col['nome'] for col in config_colunas['colunas']])
    elif caminho_completo.endswith('.bin'):
        pass
    else:
        raise ValueError("Formato de arquivo não suportado")

    print('convertendo tipos de dados')

    for col in config_colunas['colunas']:
        if col['tipo'] == 'int':
            dados[col['nome']] = dados[col['nome']].astype(int)
        elif col['tipo'] == 'float':
            dados[col['nome']] = dados[col['nome']].astype(float)
        elif col['tipo'] == 'data':
            dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

    print('conctando no banco de dados')
    banco = '/workspaces/api-geral/ING/database/banco_cep_teste.db'
    conn = sqlite3.connect(f'{banco}')
    cursor = conn.cursor()

    print('Criando tabela no banco de dados...')
    sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_estrutura['tabela']} ("
    for col in config_colunas['colunas']:
        sql_create_table += f"{col['nome']} {col['tipo']}, "
    sql_create_table = sql_create_table[:-2] + ")"
    cursor.execute(sql_create_table)

    print('Inserindo dados na tabela...')
    for index, row in dados.iterrows():
        mapped_values = []
        for col in config_colunas['colunas']:
            mapped_values.append(row.get(col['nome'], None))
        print("Valores mapeados:", mapped_values)
        sql = f"INSERT INTO {config_estrutura['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
        print("SQL:", sql)
        cursor.execute(sql, mapped_values)

    print('Commit e fechamento da conexão...')
    conn.commit()
    conn.close()

    print("Ingestão de dados concluída com sucesso!")
except FileExistsError as error:
    print(error)

lendo estrutura de coluna json
lendo estrutura de coluna json
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste', 'nomeArquivoChave': 'teste'}
palavra chave: teste
dados conteudo_load: ['resultado_teste_20240617_222901.txt\n']
Arquivo encontrado: /workspaces/api-geral/ING/app/src/data/landing/resultado_teste_20240617_222901.txt
conteudo a gravar: resultado_teste_20240617_222901.txt
dados ignorado!
convertendo tipos de dados
conctando no banco de dados
Criando tabela no banco de dados...
Inserindo dados na tabela...
Valores mapeados: ['id', 'nome']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['1', 'teste']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['2', 'teste2']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Valores mapeados: ['3', 'teste3']
SQL: INSERT INTO tabela_teste (id, nome) VALUES (?, ?)
Commit e fechamento da conexão...
Ingestão de dados concluída com sucesso!


In [ ]:
arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'



with open(file_tb_json, 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_colunas)
    key = config_colunas['nomeArquivoChave']
    print(f"palavra chave: {key}")


with open(arq_dir, 'r+') as dados:
    conteudo_load = dados.readlines()
    print(f'dados conteudo_load: {conteudo_load}')

    for root, dirs, lista in os.walk(landing):
        for nome_arquivo in lista:
            if key in nome_arquivo:
                caminho_completo = os.path.join(root, nome_arquivo)
                print(f'Arquivo encontrado: {caminho_completo}')
                sav = os.path.basename(caminho_completo)
                print(f'conteudo a gravar: {sav}')
                
                if sav + '\n' not in conteudo_load:
                    dados.write(f"{sav}\n")
                    print(f'dados salvo: {sav}')
                    print('dados gravado!')
                else:
                    print('dados ignorado!')

    
    dados.flush()
    os.ftruncate(dados.fileno(), dados.tell())

In [83]:
# list_dir = "/workspaces/api-geral/ING/app/src/data/landing/"
arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'

with open(f'{file_tb_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_colunas)
    key = config_colunas['nomeArquivoChave']
    print(f"palavra chave: {key}")


for root, dirs, lista in os.walk(landing):
    for nome_arquivo in lista:
        if key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
            print(caminho_completo)
            sav = os.path.basename(caminho_completo)
            dados = open(f"{arq_dir}", "r+")
            conteudo_load = dados.readlines()           
            print(f"dados conteudo_load: {conteudo_load}")
            save = list()
            save.append(f"{sav}")
            print(f'conteudo do save:{save}')
            print('==========================')
            if save != conteudo_load:
                dados.write(f"{save}")
                print(f'dados salvo: {save}')
                print('dados gravado!')
            else:
                print('dados ignorado!')
            # if save != counteudo_load:
            #     dados.writelines(f"{save}")
            #     print('arquivo adicionado na lista')  
            #     print(f'caminho: {arq_dir}')
            #     dados.close()
            #     print(f"append no arquivo: {save}")
            # elif dados.close():
            #      print('arquivo ignorado!')
            # else:
            #     print('código encerrado!')
                
                        
                
                    
                
                
            
            #SALVAR NOME DO ARQUIVO QUANDO ELE NÃO EXISTIR NO arquivoteste.txt E SE EXISTIR IGNORAR
            
            
            # if sav != load:
            #     save.append(sav)
            #     load.writelines(f"{save} \n")
            #     print('arquivo adicionado na lista')
            # else:
            #     print('Arquivo ignorado')
            



# for root, dirs, files in os.walk(landing):
#     for nome_arquivo in files:
#         if key in nome_arquivo:
#             caminho_completo = os.path.join(root, nome_arquivo)
#             print(caminho_completo)

lendo estrutura de coluna json
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste', 'nomeArquivoChave': 'teste'}
palavra chave: teste
/workspaces/api-geral/ING/app/src/data/landing/resultado_teste_20240617_222901.txt
dados conteudo_load: ["['resultado_teste_20240617_222901.txt']['resultado_teste_20240617_222901.txt']"]
conteudo do save:['resultado_teste_20240617_222901.txt']
dados salvo: ['resultado_teste_20240617_222901.txt']


In [164]:
list_dir = "/workspaces/api-geral/ING/app/src/data/landing/"
arq_dir = "/workspaces/api-geral/ING/app/src/data/tmp/arquivoteste.txt"
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'

with open(f'{file_tb_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_colunas)
    key = config_colunas['nomeArquivoChave']
    print(f"palavra chave: {key}")

listando_landing = os.listdir(list_dir)
print(listando_landing)

try:
    for lista in listando_landing:
        if key == lista:
            sav = os.path.basename(lista)
            load = open(f"{arq_dir}", "a")
            save = list()
            save.append(sav)
            load.writelines(f"{save} \n")
            print('arquivo adicionado na lista')
        else:
            print('arquivo não encontrado')
except FileExistsError as e:
    print(e)

for files in os.walk(landing):
    for nome_arquivo in files:
        if key in nome_arquivo:
            print(nome_arquivo)
    # if save == load:
    #     load.writelines(f"{save} \n")
    # else:
    #     print('Arquivo já ingerido')
    



lendo estrutura de coluna json
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste', 'nomeArquivoChave': 'teste'}
palavra chave: teste
['resultado_teste_20240617_222901.txt', 'resultado_concat_20240513_162447.txt']
arquivo não encontrado
arquivo não encontrado


In [155]:
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/teste/estrutura.json'

with open(f'{file_tb_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')
    print(config_colunas)
    key = config_colunas['nomeArquivoChave']
    print(f"palavra chave: {key}")

# word_key = 'teste'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print(caminho_arquivo)

lendo estrutura de coluna json
{'banco': 'banco_teste.db', 'tabela': 'tabela_teste', 'descricao': 'tabela teste cep', 'nomeArquivoChave': 'teste'}
palavra chave: teste
/workspaces/api-geral/ING/app/src/data/landing/resultado_teste_20240617_222901.txt


In [ ]:
#como pegar o nome do arquivo e gravar o nome em outro arquivo ? 

In [5]:
print('iniciando ingestão!')
time.sleep(3)

file_col_json = '/workspaces/api-geral/ING/app/src/layouts/cep/tabela_cep.json'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/cep/estrutura.json'
bckp_path = '/workspaces/api-geral/ING/app/src/bckp/'
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
word_key = 'resultado'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

print('carregando variaveis json')
time.sleep(3)
# Ler configurações das colunas
with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

time.sleep(3)
# Ler configurações da tabela
with open(f'{file_tb_json}', 'r') as f:
    config_tabela = json.load(f)
    print('lendo estrutura tabela json')

print('carregando variaveis do arquivo de ingestão')
time.sleep(3)

print('realizando iteração')
print('buscando o ultimo arquivo ingerido de acordo com o timestamp')
time.sleep(3)
for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print('arquivo encontrado!')
                print('realizando checking do arquivo.')
                time.sleep(3)
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.txt'}:
                        print(f'file extension txt, ok! extension: {file_extension}')
                    else:
                        print('extension txt error, not found')
                    print(caminho_arquivo)
                    
print('carregando arquivo...')

iniciando ingestão!
carregando variaveis json
lendo estrutura de coluna json
lendo estrutura tabela json
carregando variaveis do arquivo de ingestão
realizando iteração
buscando o ultimo arquivo ingerido de acordo com o timestamp
arquivo encontrado!
realizando checking do arquivo.
True
file extension json, ok! extension: .txt
/workspaces/api-geral/ING/app/src/data/landing/resultado_concat_20240513_162447.txt
carregando arquivo...


In [5]:
print('iniciando ingestão!')
time.sleep(3)

file_col_json = '/workspaces/api-geral/ING/app/src/layouts/cep/tabela_cep.json'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/cep/estrutura.json'
bckp_path = '/workspaces/api-geral/ING/app/src/bckp/'
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
word_key = 'resultado'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

print('carregando variaveis json')
time.sleep(3)
# Ler configurações das colunas
with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

time.sleep(3)
# Ler configurações da tabela
with open(f'{file_tb_json}', 'r') as f:
    config_tabela = json.load(f)
    print('lendo estrutura tabela json')

print('carregando variaveis do arquivo de ingestão')
time.sleep(3)

print('realizando iteração')
print('buscando o ultimo arquivo ingerido de acordo com o timestamp')
time.sleep(3)
for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print('arquivo encontrado!')
                print('realizando checking do arquivo.')
                time.sleep(3)
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.json'}:
                        print(f'file extension json, ok! extension: {file_extension}')
                    else:
                        print('extension json error, not found')
                    print(caminho_arquivo)

print('carregando arquivo...')
time.sleep(3)
# Carregar dados
arquivo_dados = caminho_arquivo
if arquivo_dados.endswith('.txt'):
    dados = pd.read_csv(arquivo_dados, delimiter=';', header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.csv'):
    dados = pd.read_csv(arquivo_dados, header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.bin'):
    # Lógica para carregar dados de um arquivo binário
    pass
else:
    raise ValueError("Formato de arquivo não suportado")

print('convertendo tipos de dados')
time.sleep(3)
# Conversão de tipos de dados
for col in config_colunas['colunas']:
    if col['tipo'] == 'int':
        dados[col['nome']] = dados[col['nome']].astype(int)
    elif col['tipo'] == 'float':
        dados[col['nome']] = dados[col['nome']].astype(float)
    elif col['tipo'] == 'data':
        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

print('conctando no banco de dados')
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()


# Criar tabela com base na estrutura definida no arquivo JSON
print('Criando tabela no banco de dados...')
sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_tabela['tabela']} ("
for col in config_colunas['colunas']:
    sql_create_table += f"{col['nome']} {col['tipo']}, "
sql_create_table = sql_create_table[:-2] + ")"
cursor.execute(sql_create_table)

# Inserir dados na tabela
print('Inserindo dados na tabela...')
for index, row in dados.iterrows():
    mapped_values = []
    for col in config_colunas['colunas']:
        mapped_values.append(row.get(col['nome'], None))
    print("Valores mapeados:", mapped_values)
    sql = f"INSERT INTO {config_tabela['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
    print("SQL:", sql)
    cursor.execute(sql, mapped_values)

# Confirmar as alterações e fechar a conexão
print('Commit e fechamento da conexão...')
conn.commit()
conn.close()

print("Ingestão de dados concluída com sucesso!")

iniciando ingestão!
carregando variaveis json
lendo estrutura de coluna json
lendo estrutura tabela json
carregando variaveis do arquivo de ingestão
realizando iteração
buscando o ultimo arquivo ingerido de acordo com o timestamp
arquivo encontrado!
realizando checking do arquivo.
True
extension json error, not found
/workspaces/api-geral/ING/app/src/data/landing/resultado_concat_20240513_162447.txt
carregando arquivo...
convertendo tipos de dados
conctando no banco de dados
Criando tabela no banco de dados...
Inserindo dados na tabela...
Valores mapeados: ['logradouro', 'bairro', 'localidade', 'uf', 'ibge', 'gia', 'ddd', 'siafi']
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
Valores mapeados: ['Rua Mairiporã', 'Mantiqueira', 'Mairiporã', 'SP', '3528502', '4339', '11', '6671']
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
Valores mapeados

In [6]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'tb_cep_brnz'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)    

Resultado:
('logradouro', 'bairro', 'localidade', 'uf', 'ibge', 'gia', 'ddd', 'siafi')
('Rua Mairiporã', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua JasminsJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua OrquideasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua GirassolJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua das RosasJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua AntúriosJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua dos CravosJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda LíriosJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda TulipasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda GiestasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda GirassolJd 